In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
4,Philly workers who got fired after raising COV...,https://www.inquirer.com/jobs/labor/covid-19-l...,"Tue, 02 Feb 2021 16:47:00 GMT",2021-02-02 16:47:00+00:00
3,Biden administration to ship COVID vaccines di...,https://www.mytwintiers.com/health/coronavirus...,"Tue, 02 Feb 2021 16:43:00 GMT",2021-02-02 16:43:00+00:00
8,"Feb. 2: Iowa surpasses 320,000 positive COVID-...",https://ktiv.com/2021/02/02/feb-2-iowa-surpass...,"Tue, 02 Feb 2021 16:22:16 GMT",2021-02-02 16:22:16+00:00
2,COVID-19 Response: Immunisation Annex - Appeal...,https://reliefweb.int/report/world/covid-19-re...,"Tue, 02 Feb 2021 16:19:00 GMT",2021-02-02 16:19:00+00:00
5,Virginia Feb. 2 COVID-19 update: Cases down ag...,https://www.wavy.com/news/health/coronavirus/v...,"Tue, 02 Feb 2021 15:34:00 GMT",2021-02-02 15:34:00+00:00
0,news Two Chiefs players clipped by barber's po...,https://www.nfl.com/news/two-chiefs-players-cl...,"Tue, 02 Feb 2021 12:07:00 GMT",2021-02-02 12:07:00+00:00
6,Latest on COVID-19 in MN: Positive trends hold...,https://www.mprnews.org/story/2021/02/02/lates...,"Tue, 02 Feb 2021 11:30:00 GMT",2021-02-02 11:30:00+00:00
1,South Carolina Ranks Fifth in Nation for COVID...,https://scdhec.gov/news-releases/south-carolin...,"Tue, 02 Feb 2021 00:57:38 GMT",2021-02-02 00:57:38+00:00
7,"Who's getting the Covid vaccination? Often, it...",https://www.nbcnews.com/health/health-news/who...,"Mon, 01 Feb 2021 21:21:00 GMT",2021-02-01 21:21:00+00:00
9,map of COVID-19 vaccination locations Mass.gov,https://www.mass.gov/info-details/covid-19-vac...,"Sun, 31 Jan 2021 16:41:15 GMT",2021-01-31 16:41:15+00:00
